In [ ]:
# Scenario 36 - 64 Beams - LiDAR! Transformers!

# Average Top-1 accuracy 0.18911290322580646
# Average Top-3 accuracy 0.29758064516129035
# Average Top-5 accuracy 0.3588709677419355
# Average Top-7 accuracy 0.42016129032258065
# Average Top-9 accuracy 0.4600806451612903
# Average Top-11 accuracy 0.5016129032258064
# Average Top-13 accuracy 0.5423387096774194
# Average Top-15 accuracy 0.5810483870967742

In [ ]:
#!pip install plyfile plotly

In [5]:
import os
import datetime
import numpy as np
import torch
import torch as t
import pandas as pd
import torch.nn as nn
import torch.cuda as cuda
import torch.optim as optimizer
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transf
from torchsummary import summary
from torch.utils.model_zoo import load_url as load_state_dict_from_url
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plyfile import PlyData, PlyElement

In [ ]:
df = pd.read_csv(r'scenario36/unit1/lidar1/lidar_frame_11-46-32.474528.csv')

# Extract the columns of interest
x = df[' X (mm)'] / 1000  # Convert to meters
y = df[' Y (mm)'] / 1000
z = df[' Z (mm)'] / 1000
reflectivity = df[' REFLECTIVITY']

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the point cloud data with color based on reflectivity
sc = ax.scatter(x, y, z, c=reflectivity, cmap='viridis', s=1)

# Add labels and a colorbar
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
cbar = fig.colorbar(sc, ax=ax)
cbar.set_label('Reflectivity')

# Show the plot
plt.show()

In [6]:
# Save directory
# year month day
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime
print(pwd)
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory)

    isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

01-08-2025
22_36
C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario36_64-Beams\Main_Folder//saved_folder//01-08-2025_22_36


In [7]:
# Data Feeding: Create data sample list
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        img_paths = row.values[1:3]
        data_samples.append(img_paths)
    return data_samples

class LidarDataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self,root_dir, num_points=15000, nat_sort = False, init_shuflle = True):
        self.num_points = num_points
        self.root = root_dir
        self.samples = create_samples(self.root,shuffle=init_shuflle,nat_sort=nat_sort)


    def __len__(self):
        return len( self.samples )

    def __getitem__(self, idx):
        sample = self.samples[idx]
        # print(sample)
        df = pd.read_csv(sample[0])
        # Extract the columns of interest
        x = df[' X (mm)'] / 1000  # Convert to meters
        y = df[' Y (mm)'] / 1000
        z = df[' Z (mm)'] / 1000
        points = np.column_stack((x.values, y.values, z.values))


        if points.shape[0] < self.num_points:
            points = np.pad(points, ((0, self.num_points - points.shape[0]), (0, 0)), mode='constant')
        elif points.shape[0] > self.num_points:
            indices = np.random.choice(points.shape[0], self.num_points, replace=False)
            points = points[indices]


        points = torch.tensor(points, dtype=torch.float32)
        label = sample[1]
        return (points.T, label)

In [10]:
num_classes=65

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class InputLayer(nn.Module):
    """Input Layer to accept input point cloud data."""
    def __init__(self, k=3):
        super(InputLayer, self).__init__()
        self.k = k

    def forward(self, x):
        # Ensure the input shape is (batch_size, num_points, channels)
        return x.transpose(1, 2)  # Change shape to (batch_size, channels, num_points)


class EmbeddingLayer(nn.Module):
    """Embedding Layer for dimensionality reduction."""
    def __init__(self, input_channels, output_channels):
        super(EmbeddingLayer, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, kernel_size=1)

    def forward(self, x):
        # x should be in shape (batch_size, channels, num_points)
        x = x.transpose(1, 2)  # Transpose to (batch_size, channels, num_points)
        return F.relu(self.conv(x))  # Applies a 1x1 convolution


class EncoderStage(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.3):
        super(EncoderStage, self).__init__()
        self.cpe = nn.Conv1d(in_channels, out_channels, kernel_size=1)  # CPE
        self.norm = nn.BatchNorm1d(out_channels)  # Normalization
        self.attention = nn.MultiheadAttention(embed_dim=out_channels, num_heads=8)  # Attention
        self.dropout_att = nn.Dropout(dropout_rate)  # Dropout after attention
        self.mlp = nn.Sequential(
            nn.Linear(out_channels, out_channels),  # MLP layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),  # Dropout after first MLP layer
            nn.Linear(out_channels, out_channels)   # Another MLP layer
        )
        self.dropout_mlp = nn.Dropout(dropout_rate)  # Dropout after second MLP layer

    def forward(self, x):
        # Apply CPE
        identity = x  # Save the input for residual connection
        x = F.relu(self.cpe(x))
        x = self.norm(x)

        # Prepare for Attention mechanism
        x_att = x.transpose(2, 1)  # Shape change for attention
        x_att, _ = self.attention(x_att, x_att, x_att)  # Attention output
        x_att = self.dropout_att(x_att)  # Apply dropout

        x_att = x_att.transpose(2, 1)  # Transpose back
        x = x + x_att  # Skip connection

        # MLP
        b, c, p = x.size()
        x = x.view(b * p, c)  # Flatten for MLP
        x = self.mlp(x)  # Apply MLP
        x = self.dropout_mlp(x)  # Apply dropout after MLP

        # Reshape back to (B, out_channels, num_points)
        output_channels = x.size(1)
        x = x.view(b, output_channels, p)
        return x


class SerializedPoolingLayer(nn.Module):
    """Serialized Pooling Layer for dimensionality reduction."""
    def __init__(self):
        super(SerializedPoolingLayer, self).__init__()

    def forward(self, x):
        return F.avg_pool1d(x, kernel_size=2)  # Average pooling


class PointTransformerV3(nn.Module):
    """Main Point Transformer V3 for Classification."""
    def __init__(self, num_classes= num_classes, num_points=15000, enc_depths=[64, 128, 256], dropout_rate=0.3):
        super(PointTransformerV3, self).__init__()

        # Input and embedding
        self.input_layer = InputLayer()
        self.embedding = EmbeddingLayer(input_channels=3, output_channels=enc_depths[0])  # Initial embedding layer

        # Encoder stages
        self.encoders = nn.ModuleList()
        in_channels = enc_depths[0]
        for out_channels in enc_depths:
            self.encoders.append(EncoderStage(in_channels, out_channels, dropout_rate=dropout_rate))
            in_channels = out_channels

        # Serialized pooling
        self.serialized_pooling = SerializedPoolingLayer()

        # Fully connected layers
        self.fc1 = nn.Linear(enc_depths[-1], 128)  # Input size from enc_depths[-1]
        self.fc2 = nn.Linear(128, 64)  # Intermediate layer
        self.fc3 = nn.Linear(64, num_classes)  # Output layer for 65 classes

    def forward(self, x):
        # Input layer
        x = self.input_layer(x)

        # Embedding layer
        x = self.embedding(x)

        # Pass through encoder stages
        for encoder in self.encoders:
            x = encoder(x)

        # Serialized pooling
        x = self.serialized_pooling(x)

        # Global feature extraction
        x = torch.max(x, dim=2)[0]  # Global max pooling across points

        x = x.view(x.size(0), -1)  # Flatten the output

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [12]:
num_classes=65
# Create an instance of the model
model = PointTransformerV3(num_classes)
# Move model to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
summary(model, (3, 15000))  # Example input shape

Layer (type:depth-idx)                   Output Shape              Param #
├─InputLayer: 1-1                        [-1, 15000, 3]            --
├─EmbeddingLayer: 1-2                    [-1, 64, 15000]           --
|    └─Conv1d: 2-1                       [-1, 64, 15000]           256
├─ModuleList: 1                          []                        --
|    └─EncoderStage: 2-2                 [-1, 64, 15000]           --
|    |    └─Conv1d: 3-1                  [-1, 64, 15000]           4,160
|    |    └─BatchNorm1d: 3-2             [-1, 64, 15000]           128
|    |    └─MultiheadAttention: 3-3      [-1, 15000, 64]           16,640
|    |    └─Dropout: 3-4                 [-1, 15000, 64]           --
|    |    └─Sequential: 3-5              [-1, 64]                  8,320
|    |    └─Dropout: 3-6                 [-1, 64]                  --
|    └─EncoderStage: 2-3                 [-1, 128, 15000]          --
|    |    └─Conv1d: 3-7                  [-1, 128, 15000]          8,320


Layer (type:depth-idx)                   Output Shape              Param #
├─InputLayer: 1-1                        [-1, 15000, 3]            --
├─EmbeddingLayer: 1-2                    [-1, 64, 15000]           --
|    └─Conv1d: 2-1                       [-1, 64, 15000]           256
├─ModuleList: 1                          []                        --
|    └─EncoderStage: 2-2                 [-1, 64, 15000]           --
|    |    └─Conv1d: 3-1                  [-1, 64, 15000]           4,160
|    |    └─BatchNorm1d: 3-2             [-1, 64, 15000]           128
|    |    └─MultiheadAttention: 3-3      [-1, 15000, 64]           16,640
|    |    └─Dropout: 3-4                 [-1, 15000, 64]           --
|    |    └─Sequential: 3-5              [-1, 64]                  8,320
|    |    └─Dropout: 3-6                 [-1, 64]                  --
|    └─EncoderStage: 2-3                 [-1, 128, 15000]          --
|    |    └─Conv1d: 3-7                  [-1, 128, 15000]          8,320


In [17]:
batch_size = 16
val_batch_size = 1
lr = 1e-2
decay = 1e-3
num_epochs = 20 # After 1st epochs, the accuracies remain same!
train_size = [1]

In [18]:
train_dir = 'scenario36_64_lidar_beam_train.csv'
val_dir = 'scenario36_64_lidar_beam_val.csv'

train_loader = DataLoader(LidarDataFeed(train_dir),
                          batch_size=batch_size,
                          #num_workers=8,
                          shuffle=False)
val_loader = DataLoader(LidarDataFeed(val_dir),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [19]:
# Training and Validation!

with cuda.device(0):

    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))

        net = PointTransformerV3(num_classes)
        net = net.cuda()

        #  Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [4,8,12], gamma=0.1, last_epoch=-1)

        count = 0
        running_loss = []
        running_top1_acc = []
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []

        best_accuracy = 0

        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            epoch_train_loss = 0  # To track the training loss for the epoch # Added
            for tr_count, (img, label) in enumerate(train_loader):
                net.train()
                x = img.cuda()
                opt.zero_grad()
                label = label.cuda()
                # print(x.shape)
                # x = torch.transpose(x, 1, 2)
                out = net.forward(x)
                L = criterion(out, label)
                L.backward()
                opt.step()
                batch_loss = L.item()
                acc_loss += batch_loss
                epoch_train_loss += batch_loss  # Accumulate batch loss for the epoch # Added
                count += 1
                if np.mod(count, 10) == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))

            epoch_train_loss /= len(train_loader)  # Calculate average training loss for the epoch # Added
            print(f'Epoch {epoch + 1} Training Loss: {epoch_train_loss:.4f}') # Added
            
            print('Start Training and Validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0
            val_loss = 0  # To track the validation loss # Added
            ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []
            gt_beam = []
            total_count = 0

            for val_count, (imgs, labels) in enumerate(val_loader):
                net.eval()
                x = imgs.cuda()
                opt.zero_grad()
                labels = labels.cuda()
                total_count += labels.size(0)
                out = net.forward(x)
                _, top_1_pred = t.max(out, dim=1)
                
                val_batch_loss = criterion(out, labels).item()  # Calculate validation loss for the batch # Added
                val_loss += val_batch_loss  # Accumulate batch loss for the epoch # Added

                gt_beam.append(labels.detach().cpu().numpy()[0])

                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
                sorted_out = t.argsort(out, dim=1, descending=True)

                top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0])

                top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0])

                top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0])

                top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0])

                top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0])

                top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0])

                top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0])

                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)

                batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
                batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
                batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
                batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
                batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
                batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
                batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
                batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()

            
            val_loss /= len(val_loader)  # Calculate average validation loss for the epoch # Added
            print(f'Epoch {epoch + 1} Validation Loss: {val_loss:.4f}') # Added
            
            print("total training examples are", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
            running_top3_acc.append(ave_top3_acc / total_count)
            running_top5_acc.append(ave_top5_acc / total_count)
            running_top7_acc.append(ave_top7_acc / total_count)
            running_top9_acc.append(ave_top9_acc / total_count)
            running_top11_acc.append(ave_top11_acc / total_count)
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)

            print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
            print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))


            cur_accuracy  = running_top1_acc[-1]

            print("current acc", cur_accuracy)
            print("best acc", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory  + '//' +  'lidar_model_64_beam'
                t.save(net.state_dict(), net_name)
                best_accuracy =  cur_accuracy
            print("updated best acc", best_accuracy)


            print("Saving the predicted value in a csv file")
            file_to_save = f'{save_directory}//topk_pred_beam_val_after_{epoch+1}th_epoch.csv'
            indx = np.arange(1, len(top1_pred_out)+1, 1)
            df1 = pd.DataFrame()
            df1['index'] = indx
            df1['link_status'] = gt_beam
            df1['top1_pred'] = top1_pred_out
            df1['top3_pred'] = top3_pred_out
            df1['top5_pred'] = top5_pred_out
            df1['top7_pred'] = top7_pred_out
            df1['top9_pred'] = top9_pred_out
            df1['top11_pred'] = top11_pred_out
            df1['top13_pred'] = top13_pred_out
            df1['top15_pred'] = top15_pred_out
            df1.to_csv(file_to_save, index=False)

            LR_sch.step()

```````````````````````````````````````````````````````
Training size is 1
Epoch No. 1
Training-Batch No.10
Loss = 3.9942381381988525
Training-Batch No.20
Loss = 3.2030186653137207
Training-Batch No.30
Loss = 4.01035737991333
Training-Batch No.40
Loss = 4.403517246246338
Training-Batch No.50
Loss = 3.208278179168701
Training-Batch No.60
Loss = 4.405714511871338
Training-Batch No.70
Loss = 4.078480243682861
Training-Batch No.80
Loss = 3.9374423027038574
Training-Batch No.90
Loss = 4.047043800354004
Training-Batch No.100
Loss = 3.8013501167297363
Training-Batch No.110
Loss = 3.9299325942993164
Training-Batch No.120
Loss = 3.915048360824585
Training-Batch No.130
Loss = 3.828789710998535
Training-Batch No.140
Loss = 3.8577306270599365
Training-Batch No.150
Loss = 4.033524036407471
Training-Batch No.160
Loss = 3.587904930114746
Training-Batch No.170
Loss = 3.413902759552002
Training-Batch No.180
Loss = 4.153591632843018
Training-Batch No.190
Loss = 3.8213536739349365
Training-Batch No.200
L

In [20]:
print('Start Validation')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
gt_beam = []
total_count = 0
            
for val_count, (imgs, labels) in enumerate(val_loader):
    net.eval()
    x = imgs.cuda()
    opt.zero_grad()
    labels = labels.cuda()
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)

    gt_beam.append(labels.detach().cpu().numpy()[0])

    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0])

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0])

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0])

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0])

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0])

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0])

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0])

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()
    
print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = np.arange(1, len(top1_pred_out)+1, 1)
df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Validation
total examples are 7440
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.18508064516129033
Average Top-3 accuracy 0.29274193548387095
Average Top-5 accuracy 0.36478494623655916
Average Top-7 accuracy 0.41935483870967744
Average Top-9 accuracy 0.4622311827956989
Average Top-11 accuracy 0.5068548387096774
Average Top-13 accuracy 0.5478494623655914
Average Top-15 accuracy 0.5865591397849462
Saving the predicted value in a csv file


In [ ]:
# Testing

In [21]:
# Load the model checkpoint
test_dir = './scenario36_64_lidar_beam_test.csv'

# Load the test data
test_data = pd.read_csv(test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

# Load the model checkpoint and prepare for evaluation
checkpoint_path = f'{checkpoint_directory}/lidar_model_64_beam'
net.load_state_dict(t.load(checkpoint_path))
net.eval()
net = net.cuda()

In [22]:
test_loader = DataLoader(LidarDataFeed(test_dir),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [23]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
gt_beam = []
total_count = 0

for val_count, (imgs, labels) in enumerate(test_loader):
    net.eval()
    x = imgs.cuda()
    opt.zero_grad()
    labels = labels.cuda()
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)

    gt_beam.append(labels.detach().cpu().numpy()[0])

    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total test examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval_Test.csv'

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
# link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
# org = test_data['original_index'].tolist()
# pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data  # Add the link_status column
df2['original_unit1_pwr3'] = pwr_60ghz # Add the original_unit1_pwr3_60ghz column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Testing
total test examples are 2480
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.18911290322580646
Average Top-3 accuracy 0.29758064516129035
Average Top-5 accuracy 0.3588709677419355
Average Top-7 accuracy 0.42016129032258065
Average Top-9 accuracy 0.4600806451612903
Average Top-11 accuracy 0.5016129032258064
Average Top-13 accuracy 0.5423387096774194
Average Top-15 accuracy 0.5810483870967742
Saving the predicted value in a csv file
